In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3

app = Flask(__name__)

# Database setup
def init_db():
    conn = sqlite3.connect('todo.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS tasks
                      (id INTEGER PRIMARY KEY, description TEXT NOT NULL, completed BOOLEAN)''')
    conn.commit()
    conn.close()

# Add task
def add_task(description):
    conn = sqlite3.connect('todo.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO tasks (description, completed) VALUES (?, ?)', (description, False))
    conn.commit()
    conn.close()

# Get tasks
def get_tasks():
    conn = sqlite3.connect('todo.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM tasks')
    tasks = cursor.fetchall()
    conn.close()
    return tasks

# Mark task as completed
def complete_task(task_id):
    conn = sqlite3.connect('todo.db')
    cursor = conn.cursor()
    cursor.execute('UPDATE tasks SET completed = TRUE WHERE id = ?', (task_id,))
    conn.commit()
    conn.close()

# Delete task
def delete_task(task_id):
    conn = sqlite3.connect('todo.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM tasks WHERE id = ?', (task_id,))
    conn.commit()
    conn.close()

# Initialize the database when the app starts
init_db()

# Home route
@app.route('/')
def index():
    tasks = get_tasks()
    return render_template('index.html', tasks=tasks)

# Add task route
@app.route('/add', methods=['POST'])
def add():
    description = request.form['description']
    add_task(description)
    return redirect(url_for('index'))

# Mark task as completed route
@app.route('/complete/<int:task_id>')
def complete(task_id):
    complete_task(task_id)
    return redirect(url_for('index'))

# Delete task route
@app.route('/delete/<int:task_id>')
def delete(task_id):
    delete_task(task_id)
    return redirect(url_for('index'))

if __name__ == "__main__":
    app.run(debug=True)
